In [2]:
import os
import sys
from pathlib import Path

In [3]:
sys.path.append(str(Path(os.path.dirname(os.getcwd())).parent))
from analysis.input_testing_data.generate_input_testing_data_for_modules import generate_testing_data
from modules.main import run_modules

## Arabidopsis analysis

In [4]:
data_directory_path = os.path.join(Path(os.getcwd()).parent.resolve(), "example_data")
genomes_gb_path = os.path.join(data_directory_path, "arabidopsis_microbiome")
zorA_file_path = os.path.join(data_directory_path, "zorA_anti_phage_defense.fasta")

In [5]:
genomes_gb_directory = Path(genomes_gb_path)
organisms_list = [str(f) for f in genomes_gb_directory.iterdir() if f.is_file()]

In [6]:
for first_organism in organisms_list:
    for second_organism in organisms_list:
        if first_organism == second_organism:
            continue
        input_dict = generate_testing_data(
            orf_optimization_method = "single_codon_diff",  # TODO - iterate over all options here
            orf_optimization_cub_index = "CAI",
            wanted_hosts = [first_organism],
            unwanted_hosts = [second_organism], 
            genome_path = genomes_gb_path,
            sequence_file_path = zorA_file_path,
            output_path = "Arabidopsis",
        )
        run_modules(input_dict, should_run_output_module=False)
        break


##########################
# User Input #
##########################
Sequence to be optimized given in the following file: /Users/shimka/PycharmProjects/Igem_TAU_2021/analysis/example_data/zorA_anti_phage_defense.fasta
Open reading frame sequence for optimization is:
ATGACCGATAGCCTTAATCTTTCCAGTTTATGGCCAGACTTGAGCAGTCTAAATGTAATGCCACCTCAAACTCCTGAGCAATTGAGTGCTTTGTTTGTGGTTATTTTATGGGGTGTCGCTCTATTTTTCTTAATTTGGTCAGTTGCTGCATTTATTCGCGCAAGACAACGAGTAGTTTGGCTTAACAAATCTTTGGATGATGCGGAGAAATCGACTCTGTCAGGTGTTCGAAATGATTTAATTTCTAGAGCTGAACAAAAAAAAGATTCAGTAGGGCATTTATGGCTCGAGTTTGATGAAACATTGCTTGAAGTAAAAGGTGCTGATGACGTTGTTCGATTGCATAACACATTTGATGCTGACTATTTTTTTAATAGTTCTAGTTTAGCTGGTGGTAGTACTGAAAACAGAATGATTGCGGTGGTTCCAGGCTTTTTAACGGCTCTGGGTGTTATTGGTACTTTTGTCGGTTTACAGCTAGGACTTTCAGATCTCAATATCGCGGGCAACGTTGATGTCAATGAAATGAAAAATGGCGTTGCTGGAGTTATAAATGGTGCAAAAATCGCATTTATGACATCGGTTTGGGGCGTGCTACTCAGTGTTGCATTTAATTTCATTGAGAAGATACTTGAGCAAATTATTAGAAAGAAAATTAAGTCTTTGCAAAACCGTATTGACAGGATGTTCCCTCGATTAAGTGCGGAATATCAACTGCAATCTATTGCAAA

### Liyam's Legacy Analysis code 

In [25]:
final_tested_org = not_really_small_genome
# func_options = ['single_codon_global', 'single_codon_local', 'zscore_hill_climbing_average', 'zscore_hill_climbing_weakest_link']
func_options = ['single_codon_global', 'zscore_hill_climbing_average']

spearman_dict = {}
for translation_function in func_options:
    opt_scores = []
    msa_scores = []

    tic = time.time()
    for org1 in final_tested_org:
        for org2 in final_tested_org:
            # if org1.split(' ')[0] == org2.split(' ')[0]:
            #     continue
            if org1 == org2:
                continue
            org_dict[org1]['optimized'] = True
            org_dict[org1]['tai_profile'] = {}
            org_dict[org1]['tai_std'] = {}
            org_dict[org1]['tai_avg'] = {}
            org_dict[org2]['optimized'] = False
            org_dict[org2]['tai_profile'] = {}
            org_dict[org2]['tai_std'] = {}
            org_dict[org2]['tai_avg'] = {}
            software_dict = {
                'sequence': cds,
                'tuning_param': 0.5,
                'organisms': {},
                }
            software_dict['organisms'][org1] = org_dict[org1]
            software_dict['organisms'][org2] = org_dict[org2]
            inner_tic = time.time()
            final_cds, optimization_index, weakest_score = run_orf_optimization(software_dict)
            print('TIME: ', time.time()-inner_tic)
            # alignment_score = pairwise2.align.globalxx(
            #     ribosomal_dict[org1], ribosomal_dict[org2], score_only=True)
            # aln_scores.append(alignment_score)
            msa_score = diff_letters(ribosomal_msa_dict[org1], ribosomal_msa_dict[org2])
            msa_scores.append(msa_score)
            opt_scores.append(optimization_index)
    spearman_dict[translation_function] = spearmanr(msa_scores, opt_scores, nan_policy='omit')
    print(spearmanr(msa_scores, opt_scores, nan_policy='omit'))
    trans_func_name = translation_function.replace('_', ' ')
    plt.scatter(msa_scores, opt_scores, s=0.1)

            # print(org1, org2, msa_scores, optimization_index)
    toc = time.time()
    print(toc-tic)

print(spearman_dict)
plt.legend(['single codon optimization', 'hill climbing optimization'], loc ="upper right")
plt.title(f'Evolutionary distance and model performance')
plt.xlabel('# Different aligned characters')
plt.ylabel('Optimization score')
plt.show()


/Users/shimka/PycharmProjects/Igem_TAU_2021/analysis/example_data/arabidopsis_microbiome
